<a href="https://colab.research.google.com/github/elliotlt/shopify/blob/main/prettify_GTM_JSON_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Document your Google Tag Manager container automatically**

### How to use this notebook



1.   [Generate an export of a GTM container](https://support.google.com/tagmanager/answer/6106997?hl=en#:~:text=to%20multiple%20containers.-,Export%20a%20container,-In%20the%20top)
2.   Generate a API key in OpenAI and paste the key in the code where it says # ⬅️ Replace with your actual API key  
3.   Run the code cell below, it will ask yu to upload your GTM container in JSON format, then the code will export your container to Google Sheet while Chat GPT documents all your tags.



### Imports and authentication

In [1]:
# --- INSTALL REQUIRED PACKAGES ---
!pip install --upgrade openai gspread oauth2client --quiet

# --- IMPORTS ---
import json
import openai
import gspread
from google.colab import files
from google.auth import default
from google.colab import auth
from gspread_dataframe import set_with_dataframe
import pandas as pd

# --- AUTHENTICATE GOOGLE SERVICES ---
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- SETUP OPENAI ---
openai_api_key = ""  # ⬅️ Replace with your actual API key
client = openai.OpenAI(api_key=openai_api_key)

# --- UPLOAD GTM JSON FILE ---
upload = files.upload()
filename = list(upload.keys())[0]
with open(filename, 'r') as file:
    gtm_data = json.load(file)

# --- LOOKUP TABLES ---
tag_types = {
    'gaawe':'GA4 Event', 'sp':'Google Ads Remarketing', 'gclidw':'Conversion Linker',
    'ua':'Universal Analytics', 'html':'Custom HTML', 'img':'Custom Image',
    'googtag': 'Google Tag', 'hjtc':'Hotjar', 'pntr':'Pinterest', 'bzi':'LinkedIn Insight',
    'qpx':'Quora Pixel', 'flc':'Floodlight Counter', 'fls':'Floodlight Sales'
}

variable_types = {
    'aev':'Auto-Event Variable', 'c':'Constant', 'j':'JavaScript Variable',
    'u':'URL', 'v':'DataLayer Variable', 'smm':'Lookup Table', 'remm':'Regex Table'
}

built_in_triggers = {
    '2147479553':'All Pages', '2147479572':'Consent Initialization'
}

# --- EXTRACT GTM ELEMENTS ---
tags = gtm_data['containerVersion'].get('tag', [])
triggers = gtm_data['containerVersion'].get('trigger', [])
variables = gtm_data['containerVersion'].get('variable', [])

# --- MAPPINGS ---
trigger_id_to_name = {t['triggerId']: t['name'] for t in triggers}
trigger_id_to_name.update(built_in_triggers)

# --- AI COMMENT GENERATOR ---
def get_ai_comment_tag(tag_name, variable_list, trigger_list):
    prompt = (
        f"What does the GTM tag '{tag_name}' do? "
        f"It uses these variables: {', '.join(variable_list)} and is triggered by: {', '.join(trigger_list)}. "
        "Please explain what it does, why it's useful, and where the data is sent in max 150 words, keep it simple"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error with tag {tag_name}: {e}")
        return "Comment generation failed"

def get_ai_comment_variable(variable_name):
    prompt = (
        f"What data does the GTM variable '{variable_name}' collect? "
        "Explain what it collects, why it's needed, and how it's used in the GTM container in simple language, max 150 words."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error with variable {variable_name}: {e}")
        return "Comment generation failed"

def get_ai_comment_trigger(trigger_name):
    prompt = (
        f"What does the GTM trigger '{trigger_name}' listen for? "
        "Explain what kind of events it fires, why it's useful, and what actions it triggers in simple language, max 150 words."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error with trigger {trigger_name}: {e}")
        return "Comment generation failed"

# --- CREATE TAGS OVERVIEW DATAFRAME ---
tag_rows = []
for tag in tags:
    name = tag.get('name', 'Unnamed Tag')
    type_ = tag_types.get(tag.get('type', ''), tag.get('type', 'Unknown'))

    # Firing Triggers
    trigger_names = [trigger_id_to_name.get(tid, tid) for tid in tag.get('firingTriggerId', [])]
    trigger_bullets = "\n• " + "\n• ".join(trigger_names) if trigger_names else ""

    # Variables used
    tag_params = tag.get('parameter', [])
    variable_names = []
    for param in tag_params:
        val = param.get('value')
        if isinstance(val, str) and val.startswith("{{") and val.endswith("}}"):
            variable_names.append(val.strip("{{}}"))
    variable_bullets = "\n• " + "\n• ".join(variable_names) if variable_names else ""

    comment = get_ai_comment_tag(name, variable_names, trigger_names)
    tag_rows.append([name, variable_bullets, trigger_bullets, comment])

tags_df = pd.DataFrame(tag_rows, columns=["Tag Name", "Variables", "Triggers", "Comment"])

# --- CREATE VARIABLES DATAFRAME ---
var_rows = []
for var in variables:
    var_name = var.get('name', 'Unnamed Variable')
    comment = get_ai_comment_variable(var_name)
    var_rows.append([
        var_name,
        variable_types.get(var.get('type', ''), var.get('type', 'Unknown')),
        comment
    ])
variables_df = pd.DataFrame(var_rows, columns=["Variable Name", "Type", "Comment"])

# --- CREATE TRIGGERS DATAFRAME ---
trigger_rows = []
for trig in triggers:
    name = trig.get('name', 'Unnamed Trigger')
    type_ = trig.get('type', 'Unknown')
    comment = get_ai_comment_trigger(name)
    trigger_rows.append([name, type_, comment])

triggers_df = pd.DataFrame(trigger_rows, columns=["Trigger Name", "Type", "Comment"])

# --- CREATE GOOGLE SHEET ---
spreadsheet = gc.create("GTM Container Documentation")
sheet_tags = spreadsheet.add_worksheet(title="Tags Overview", rows="100", cols="20")
sheet_vars = spreadsheet.add_worksheet(title="Variables", rows="100", cols="10")
sheet_trigs = spreadsheet.add_worksheet(title="Triggers", rows="100", cols="10")

# Delete default empty sheet
default_sheet = spreadsheet.sheet1
spreadsheet.del_worksheet(default_sheet)

# --- WRITE TO SHEETS ---
set_with_dataframe(sheet_tags, tags_df)
set_with_dataframe(sheet_vars, variables_df)
set_with_dataframe(sheet_trigs, triggers_df)

# --- DONE ---
print(f"✅ Google Sheet created: https://docs.google.com/spreadsheets/d/{spreadsheet.id}/edit")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 15.0 MB/s eta 0:00:00


Saving GTM-KFSPMCLV_workspace32.json to GTM-KFSPMCLV_workspace32.json
Error with tag CMP - Cookiebot Banner: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error with tag GTAG - Config tag: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
Error with tag GA4 - All Events: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/